In [1]:
from ipywidgets import interact, interactive, IntSlider, Layout
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import pandas as pd

In [2]:
import pymysql
import geopip

In [3]:
from sklearn.linear_model import logistic
from sklearn.model_selection import train_test_split

In [8]:
def func(latitude, longitude):
    connection = pymysql.connect(host='localhost',
                             database='nasadata',
                             user='root',
                             passwd='felipe3211')

    mycursor = connection.cursor()
    
    sql = "SELECT * from irradiation2 WHERE latitude =%s AND longitude =%s"
    
    try:
        response = geopip.search(lat=latitude, lng=longitude)
        country = response.get('NAME')
        
        if country != 'Brazil':
            exit("Coords do not belong to Brazil!")
    except:
        exit("Coords do not belong to Brazil!")

    # Process latitude and longitude to do reguest to database
    rounded_lat = round(latitude, 1)
    rounded_lng = round(longitude, 1)

    # Bellow I Have decimal parte of earch number
    decimal_part_of_lat = round(rounded_lat-int(rounded_lat), 1)
    decimal_part_of_lng = round(rounded_lng-int(rounded_lng), 1)

    # Verify if I have the latitude in DB
    increment_lat = 0
    increment_lng = 0
    if decimal_part_of_lat*10 % 2 == 0:
        pass
    else:
        increment_lat = 0.1

    if decimal_part_of_lng*10 % 2 == 0:
        pass
    else:
        increment_lng = 0.1

    lat_query = round(rounded_lat+increment_lat, 1)
    lng_query = round(rounded_lng+increment_lng, 1)

    val = (lat_query, lng_query)
    mycursor.execute(sql, val)
    records = mycursor.fetchall()
# todo insert here tratment to case record are empty
    list_of_months = [[], [], [], [], [], [], [], [], [], [], [], []]  # List 12 X N
    for row in records:
        identify, lat, lng, year, month, value = row
        if value == -999.0:  # append only valid values
            continue
        list_of_months[int(month)-1].append(float(value))
        month_averages = []
        
    for i in range(12):
        average = sum(list_of_months[i])/len(list_of_months[i])
        month_averages.append((float(average)))
        #print("Para o mes {} a media foi {}".format(i+1, average))
        
        # Set the locator
    locator = mdates.MonthLocator
    # Specify the format - %b gives up Jan, Feb...
    fmt = mdates.DateFormatter('%b')
    month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
                  'August', 'September', 'October', 'November', 'December']
    
    font1 = {
    'weight': 'bold'
    }

    # plot area
    plt.figure()
    fix, ax = plt.subplots(1, 1)

    # todo parametrise values of x and y with base of max and min value of irradiation
    #plt.text(x=0, y=3, s="Latitude: ", fontdict=font1)
    #plt.text(x=2, y=3, s=str(latitude))
    #plt.text(x=0, y=2.5, s="Longitude: ", fontdict=font1)
    #plt.text(x=2, y=2.5, s=str(longitude))

    ax.plot(month_list, month_averages, 'o-')

    plt.xticks(rotation=-45)
    plt.ylabel('kW-hr/m^2/day')
    plt.grid(True)

    plt.title('All Sky Insolation Average per Month of lat = {} lon = {}'.format(str(latitude), str(longitude)))
    plt.rcParams['figure.figsize'] = [10, 7]
    plt.show()


In [9]:
pl=interactive(func, latitude=(-34,4,1), longitude=(-74,-33,1))
display(pl)

interactive(children=(IntSlider(value=-15, description='latitude', max=4, min=-34), IntSlider(value=-54, descr…